In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import datetime
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core import Experiment
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies

import collections
from sklearn.model_selection import train_test_split
import pickle as pkl
import logging
import azureml.dataprep as dprep 
from azureml.core import Dataset
# from azureml.data.tabular_dataset import TabularDataset

In [4]:
#load the data
data_df = pd.read_csv('data/ML_Testing_data_Risk_Sales.csv', low_memory=False)
test_df= pd.read_csv('data/ML_Training_data_Risk_Sales.csv', low_memory=False)

In [5]:
data_df.fillna(data_df.mean(), inplace=True)

In [6]:
data_df.head()

,SALES_CHANNEL,TOTAL_PRICE,MAKE1,MODEL1,PRICE1,MAKE2,MODEL2,PRICE2,MAKE3,MODEL3,...,FRAUDSTATUSIND,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label
0,TELESALES,847.92,Apple,iPhone XR,649.99,NaN,NaN,NaN,NaN,NaN,...,Yellow,18.65,21.18,7.83,8284.93,2854.88,6020.49,2820.68,1,Fraud
1,TELESALES,0.00,Apple,iPhone 7 Plus,669.99,NaN,NaN,NaN,NaN,NaN,...,Green,20.32,93.03,8.30,11056.48,4173.41,6020.49,4099.41,0,Good
2,TELESALES,126.96,Motorola,moto e5 play,119.99,NaN,NaN,NaN,NaN,NaN,...,Green,22.80,22.65,1.77,8048.73,4173.41,6020.49,4099.41,0,Good
3,RETAIL,655.26,Samsung,Galaxy S9,599.99,NaN,NaN,NaN,NaN,NaN,...,Green,22.80,10.97,1467.63,7060.18,4173.41,6020.49,4099.41,0,Good
4,ONLINE,479.57,Apple,iPhone 7,449.99,NaN,NaN,NaN,NaN,NaN,...,Green,18.95,43.45,59000.28,4889.35,4173.41,6020.49,4099.41,0,Good


In [7]:
data_df.tail()

,SALES_CHANNEL,TOTAL_PRICE,MAKE1,MODEL1,PRICE1,MAKE2,MODEL2,PRICE2,MAKE3,MODEL3,...,FRAUDSTATUSIND,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label
36992,TELESALES,421.64,Motorola,moto g7 power,239.99,Motorola,moto g7 power,239.99,OtterBox,moto g⁷ power Commuter Lite,...,Yellow,28.87,41.23,5676.42,8284.93,2395.85,6020.49,2325.57,1,Fraud
36993,ONLINE,2055.98,Apple,iPhone XS Max,999.99,Case-Mate,Apple iPhone XS Max Case & GLASS Screen Protec...,44.95,Apple,iPhone XR,...,Red,22.80,15.40,1274.25,21072.83,6503.03,14569.80,6480.05,1,Fraud
36994,TELESALES,1626.38,Samsung,Galaxy S10,899.99,LG,G8 ThinQTM,779.99,Case-Mate,Samsung Galaxy S10 Barely There,...,Red,22.80,12.12,102752.05,8284.93,4113.40,6020.49,4087.20,1,Fraud
36995,TELESALES,1176.68,Apple,iPhone XR,599.99,Apple,iPhone 7 Plus (PRODUCT)RED,469.99,Samsung,Galaxy A50,...,Fraud Check Passed,22.80,12.03,16.12,10732.88,4146.32,6586.57,4116.28,0,Good
36996,TELESALES,995.76,Samsung,Galaxy A50,349.99,Samsung,Galaxy A50,349.99,Motorola,moto e5 play,...,Fraud Check Passed,22.80,25.72,3.57,15757.08,3633.18,12123.90,3588.18,0,Good


In [8]:
data=data_df[['SALES_CHANNEL','TOTAL_PRICE','IDA_RESULT','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','START_TO_CLOSE_MINS','START_TO_CASECLOSE_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','ORDER_RISK_LABEL','Risk Label']]
test_data=test_df[['SALES_CHANNEL','TOTAL_PRICE','IDA_RESULT','FINAL_RESULT','HOUR_OF_DAY','ACTIVE_WATCH_DURATION_MINS','ORDER_HOLD_DURATION_MINS','ACCOUNT_OPEN_TO_ORDER_MINS','START_TO_CLOSE_MINS','START_TO_CASECLOSE_MINS','CASECLOSE_TO_ORDERCLOSE_MINS','CASEOPEN_TO_CASECLOSE_MINS','ORDER_RISK_LABEL','Risk Label']]

In [9]:
data.shape

(36997, 14)

In [10]:
test_data.shape

(96458, 14)

In [11]:
data.head()

,SALES_CHANNEL,TOTAL_PRICE,IDA_RESULT,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label
0,TELESALES,847.92,GREEN,YELLOW,16,18.65,21.18,7.83,8284.93,2854.88,6020.49,2820.68,1,Fraud
1,TELESALES,0.00,GREEN,GREEN,23,20.32,93.03,8.30,11056.48,4173.41,6020.49,4099.41,0,Good
2,TELESALES,126.96,GREEN,GREEN,1,22.80,22.65,1.77,8048.73,4173.41,6020.49,4099.41,0,Good
3,RETAIL,655.26,GREEN,GREEN,19,22.80,10.97,1467.63,7060.18,4173.41,6020.49,4099.41,0,Good
4,ONLINE,479.57,GREEN,GREEN,11,18.95,43.45,59000.28,4889.35,4173.41,6020.49,4099.41,0,Good


In [12]:
data.tail()

,SALES_CHANNEL,TOTAL_PRICE,IDA_RESULT,FINAL_RESULT,HOUR_OF_DAY,ACTIVE_WATCH_DURATION_MINS,ORDER_HOLD_DURATION_MINS,ACCOUNT_OPEN_TO_ORDER_MINS,START_TO_CLOSE_MINS,START_TO_CASECLOSE_MINS,CASECLOSE_TO_ORDERCLOSE_MINS,CASEOPEN_TO_CASECLOSE_MINS,ORDER_RISK_LABEL,Risk Label
36992,TELESALES,421.64,GREEN,YELLOW,3,28.87,41.23,5676.42,8284.93,2395.85,6020.49,2325.57,1,Fraud
36993,ONLINE,2055.98,GREEN,RED,23,22.80,15.40,1274.25,21072.83,6503.03,14569.80,6480.05,1,Fraud
36994,TELESALES,1626.38,GREEN,RED,19,22.80,12.12,102752.05,8284.93,4113.40,6020.49,4087.20,1,Fraud
36995,TELESALES,1176.68,GREEN,YELLOW,15,22.80,12.03,16.12,10732.88,4146.32,6586.57,4116.28,0,Good
36996,TELESALES,995.76,GREEN,RED,1,22.80,25.72,3.57,15757.08,3633.18,12123.90,3588.18,0,Good


In [15]:
data['START_TO_CLOSE_MINS'].fillna('NA',inplace=True)
data['START_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)
data['CASECLOSE_TO_ORDERCLOSE_MINS'].fillna('NA',inplace=True)
data['CASEOPEN_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)


test_data['START_TO_CLOSE_MINS'].fillna('NA',inplace=True)
test_data['START_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)
test_data['CASECLOSE_TO_ORDERCLOSE_MINS'].fillna('NA',inplace=True)
test_data['CASEOPEN_TO_CASECLOSE_MINS'].fillna('NA',inplace=True)

In [16]:
#linear regression
Y = data_df['START_TO_CLOSE_MINS']
X = data_df['RiskLabel']

KeyError: 'RiskLabel'